<a href="https://colab.research.google.com/github/kevnantony/TSPrediction/blob/main/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pystan==2.19.1.1 --quiet
!pip install prophet --quiet
!pip install yfinance --quiet

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
ERROR: Could not build wheels for pystan, which is required to install pyproject.toml-based projects
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
ERROR: Could not build wheels for pystan, which is required to install pyproject.toml-based projects


In [6]:
!pip install localpip
!localpip install prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=17f1a692a5162e0e4bec5ef34fcdc115f5832b32f8cb20f8b7006e35e28688a5
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
Traceback (most recent call last):
  File "/usr/local/bin/localpip", line 5, in <module>
    from localpip.__main__ import main
  File "/usr/local/lib/python3.10/dist-packages/localpip/__main__.py", line 5, in <module>
    from ctypes import Structure, byref, windll
ImportError: cannot import name 'windll' from 'ctypes' (/usr/lib/python3.10/ctypes/__init__.py)


In [7]:
!pip install plotly


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [24]:
today = '2022-12-31'
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-26,"$1,218.92","$1,226.97","$1,214.34","$1,226.97","$1,226.97",3282098400
2022-12-27,"$1,226.99","$1,230.42","$1,205.90","$1,212.79","$1,212.79",4091530737
2022-12-28,"$1,212.74","$1,213.13","$1,185.70","$1,189.99","$1,189.99",4991669631
2022-12-29,"$1,190.01","$1,204.14","$1,188.36","$1,201.60","$1,201.60",4132233940
2022-12-30,"$1,201.57","$1,202.03","$1,187.46","$1,199.23","$1,199.23",4055668253


In [25]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1878 entries, 2017-11-09 to 2022-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1878 non-null   float64
 1   High       1878 non-null   float64
 2   Low        1878 non-null   float64
 3   Close      1878 non-null   float64
 4   Adj Close  1878 non-null   float64
 5   Volume     1878 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 102.7 KB


In [26]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [27]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [28]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [29]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [30]:
df.tail()

,ds,y
1873,2022-12-26,"$1,218.92"
1874,2022-12-27,"$1,226.99"
1875,2022-12-28,"$1,212.74"
1876,2022-12-29,"$1,190.01"
1877,2022-12-30,"$1,201.57"


In [31]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [32]:
m = Prophet(
    seasonality_mode="multiplicative"
)

m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptck07m2x/op8xkg79.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptck07m2x/u8vlvh6x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=61277', 'data', 'file=/tmp/tmptck07m2x/op8xkg79.json', 'init=/tmp/tmptck07m2x/u8vlvh6x.json', 'output', 'file=/tmp/tmptck07m2x/prophet_model0jboexvn/prophet_model-20230621134859.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:48:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:49:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [33]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2238,2023-12-26
2239,2023-12-27
2240,2023-12-28
2241,2023-12-29
2242,2023-12-30


In [34]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2238,2023-12-26,"$-1,764.49","$-3,417.54",$-62.25
2239,2023-12-27,"$-1,771.06","$-3,483.11",$-80.62
2240,2023-12-28,"$-1,784.95","$-3,464.66",$-133.71
2241,2023-12-29,"$-1,783.52","$-3,493.45",$-41.12
2242,2023-12-30,"$-1,776.71","$-3,484.17",$-75.28


In [35]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

-185.34402595528164

In [36]:
plot_plotly(m, forecast)

In [37]:
plot_components_plotly(m, forecast)